In [1]:
import qiskit as qs

In [2]:
qs.IBMQ.load_account()

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [3]:
provider = qs.IBMQ.get_provider(hub='ibm-q')
for backend in provider.backends():
    print(backend.status().to_dict())

ibmq_16_melbourne = provider.get_backend('ibmq_16_melbourne')

{'backend_name': 'ibmq_qasm_simulator', 'backend_version': '0.1.547', 'operational': True, 'pending_jobs': 2, 'status_msg': 'active'}
{'backend_name': 'ibmqx2', 'backend_version': '2.2.0', 'operational': True, 'pending_jobs': 30, 'status_msg': 'active'}
{'backend_name': 'ibmq_16_melbourne', 'backend_version': '2.3.1', 'operational': True, 'pending_jobs': 21, 'status_msg': 'active'}
{'backend_name': 'ibmq_vigo', 'backend_version': '1.3.0', 'operational': True, 'pending_jobs': 72, 'status_msg': 'active'}
{'backend_name': 'ibmq_ourense', 'backend_version': '1.3.0', 'operational': True, 'pending_jobs': 51, 'status_msg': 'active'}
{'backend_name': 'ibmq_valencia', 'backend_version': '1.4.0', 'operational': True, 'pending_jobs': 11, 'status_msg': 'active'}
{'backend_name': 'ibmq_armonk', 'backend_version': '1.1.1', 'operational': True, 'pending_jobs': 10, 'status_msg': 'active'}
{'backend_name': 'ibmq_athens', 'backend_version': '1.2.0', 'operational': True, 'pending_jobs': 14, 'status_msg':

In [4]:
properties = ibmq_16_melbourne.properties()
coupling_map = ibmq_16_melbourne.configuration().coupling_map
print("properties: ", properties)
print("coupling_map: ", coupling_map)
# From this we can construct a noise model to mimic the noise on the device (we will discuss noise models further later in the textbook).

properties:  <qiskit.providers.models.backendproperties.BackendProperties object at 0x106e76f70>
coupling_map:  [[0, 1], [0, 14], [1, 0], [1, 2], [1, 13], [2, 1], [2, 3], [2, 12], [3, 2], [3, 4], [3, 11], [4, 3], [4, 5], [4, 10], [5, 4], [5, 6], [5, 9], [6, 5], [6, 8], [7, 8], [8, 6], [8, 7], [8, 9], [9, 5], [9, 8], [9, 10], [10, 4], [10, 9], [10, 11], [11, 3], [11, 10], [11, 12], [12, 2], [12, 11], [12, 13], [13, 1], [13, 12], [13, 14], [14, 0], [14, 13]]


In [5]:
ibmq_16_melbourne_noise_model = qs.providers.aer.noise.NoiseModel.from_backend(properties)
# And then run the job on the emulator, with it reproducing all these features of the real device. Here's an example with a circuit that should output '10' in the noiseless case.

In [7]:
q_circuit = qs.QuantumCircuit(2, 2)
q_circuit.x(1)
q_circuit.measure(0, 0)
q_circuit.measure(1, 1)
q_circuit.draw()

┌─┐   
q_0: ─────┤M├───
     ┌───┐└╥┘┌─┐
q_1: ┤ X ├─╫─┤M├
     └───┘ ║ └╥┘
c: 2/══════╩══╩═
           0  1

In [10]:
qasm_simulator = qs.Aer.get_backend("qasm_simulator")
job_1 = qs.execute( \
    q_circuit, \
    qasm_simulator, \
    shots=1024, \
    noise_model = ibmq_16_melbourne_noise_model, \
    coupling_map = coupling_map, \
    basis_gates=ibmq_16_melbourne_noise_model.basis_gates)
result_1 = job_1.result()

In [11]:
print(result_1.get_counts())

{'00': 60, '10': 961, '11': 3}


In [12]:
qasm_simulator = qs.Aer.get_backend("qasm_simulator")
job_2 = qs.execute(q_circuit, qasm_simulator, shots=1024)
result_2 = job_2.result()

In [13]:
print(result_2.get_counts())

{'10': 1024}
